# Building an optical character recognition engine

In [1]:
import numpy as np
import neurolab as nl

# Define the input file
input_file = 'letter.data'

# Define the number of datapoints to 
# be loaded from the input file
num_datapoints = 50

# String containing all the distinct characters
orig_labels = 'onamgid'

# Compute the number of distinct characters
num_orig_labels = len(orig_labels)

# Define the training and testing parameters
num_train = int(0.9 * num_datapoints)
num_test = num_datapoints - num_train

In [2]:
# Define the dataset extraction parameters 
start = 6
end = -1

# Creating the dataset
data = []
labels = []
with open(input_file, 'r') as f:
    for line in f.readlines():
        # Split the current line tabwise
        list_vals = line.split('\t')

        # Check if the label is in our ground truth 
        # labels. If not, we should skip it.
        if list_vals[1] not in orig_labels:
            continue

        # Extract the current label and append it 
        # to the main list
        label = np.zeros((num_orig_labels, 1))
        label[orig_labels.index(list_vals[1])] = 1
        labels.append(label)

        # Extract the character vector and append it to the main list
        cur_char = np.array([float(x) for x in list_vals[start:end]])
        data.append(cur_char)

        # Exit the loop once the required dataset has been created 
        if len(data) >= num_datapoints:
            break

# Convert the data and labels to numpy arrays
data = np.asfarray(data)
labels = np.array(labels).reshape(num_datapoints, num_orig_labels)

In [3]:
data.shape

(50, 128)

In [4]:
labels.shape

(50, 7)

In [5]:
# Extract the number of dimensions
num_dims = len(data[0])
num_dims

128

In [6]:
# Create a feedforward neural network
nn = nl.net.newff([[0, 1] for _ in range(len(data[0]))], 
        [128, 16, num_orig_labels])

# Set the training algorithm to gradient descent
nn.trainf = nl.train.train_gd

# Train the network
error_progress = nn.train(data[:num_train,:], labels[:num_train,:], 
        epochs=10000, show=500, goal=0.01)

Epoch: 500; Error: 3.0106654820359284;
Epoch: 1000; Error: 2.2262325536399032;
Epoch: 1500; Error: 1.9921171008221803;
Epoch: 2000; Error: 0.6290572023278154;
Epoch: 2500; Error: 0.074028031348779;
Epoch: 3000; Error: 0.04607612243635758;
Epoch: 3500; Error: 0.030781820555644022;
Epoch: 4000; Error: 0.016536390883293735;
Epoch: 4500; Error: 0.03269958593408773;
Epoch: 5000; Error: 0.03375918814704432;
Epoch: 5500; Error: 0.013802365023049856;
The goal of learning is reached


In [7]:
# Predict the output for test inputs 
print('\nTesting on unknown data:')
predicted_test = nn.sim(data[num_train:, :])
for i in range(num_test):
    print('\nOriginal:', orig_labels[np.argmax(labels[i])])
    print('Predicted:', orig_labels[np.argmax(predicted_test[i])])


Testing on unknown data:

Original: o
Predicted: o

Original: m
Predicted: n

Original: m
Predicted: a

Original: a
Predicted: a

Original: n
Predicted: n
